# Building Your Own LLM Agent for Gomoku: A Step-by-Step Tutorial

Welcome to this hands-on tutorial where you'll learn how to create your own AI agent that uses a Large Language Model (LLM) to play Gomoku (also known as Five-in-a-Row)!

## What You'll Learn
- How to design and implement an AI agent from scratch
- How Large Language Models can be used for strategic gameplay
- How to integrate your agent with an existing game framework
- How to run competitions between different AI strategies

## Prerequisites
- Basic Python knowledge
- Understanding of classes and inheritance
- Familiarity with async/await (we'll explain as we go)

Let's get started!

In [1]:
!pip install git+https://github.com/sitfoxfly/gomoku-ai

  Cloning https://github.com/sitfoxfly/gomoku-ai to c:\users\alexander willy joha\appdata\local\temp\pip-req-build-yp_gn8m6
  Resolved https://github.com/sitfoxfly/gomoku-ai to commit 13efdaa4ad34f4c42fe600998528c6d2c09e63de
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'


  Running command git clone --filter=blob:none --quiet https://github.com/sitfoxfly/gomoku-ai 'C:\Users\ALEXANDER WILLY JOHA\AppData\Local\Temp\pip-req-build-yp_gn8m6'


## Understanding the Game Framework

First, let's explore the existing codebase to understand how agents work in this Gomoku framework.

In [3]:
# Import the necessary modules
import random
from typing import Tuple

# Import the game framework components
from gomoku.agents.base import Agent
from gomoku.core.models import GameState, Player
from gomoku.arena import GomokuArena
from gomoku.utils import ColorBoardFormatter
from gomoku.llm.openai_client import OpenAIGomokuClient

Agent Base Class Structure:
========================================

Every agent must implement the 'get_move()' method!  This method receives the current game state and returns a (row, col) tuple.

📍 `get_move(game_state: GameState)`: Return (row, col) for next move.

📍 `_setup()`: Initializes the agent (e.g., OpenAI client, system prompt, etc.)



## Understanding the Game State

Before we create our agent, let's understand what information the `GameState` provides:

In [6]:
# Create a sample game state to explore

from gomoku.core.game_logic import GomokuGame

# Initialize a small game for demonstration
demo_game = GomokuGame(board_size=8)
demo_state = demo_game.state

print("📋 GameState Information Available to Your Agent:")
print("=" * 50)
print(f"🔹 Board size: {demo_state.board_size}x{demo_state.board_size}")
print(f"🔹 Current player: {demo_state.current_player.value}")
print(f"🔹 Move count: {len(demo_state.move_history)}")
print(f"🔹 Legal moves available: {len(demo_state.get_legal_moves())}")

print("\n📊 Board Layout:")
print(demo_state.format_board('standard'))

print("\n💡 Key Methods You Can Use:")
print("   • is_valid_move(row, col) - Check if a move is legal")
print("   • get_legal_moves() - Get all available positions")
print("   • format_board(formatter) - Get board as string for LLM")

📋 GameState Information Available to Your Agent:
🔹 Board size: 8x8
🔹 Current player: X
🔹 Move count: 0
🔹 Legal moves available: 64

📊 Board Layout:
    0  1  2  3  4  5  6  7 
 0  .  .  .  .  .  .  .  . 
 1  .  .  .  .  .  .  .  . 
 2  .  .  .  .  .  .  .  . 
 3  .  .  .  .  .  .  .  . 
 4  .  .  .  .  .  .  .  . 
 5  .  .  .  .  .  .  .  . 
 6  .  .  .  .  .  .  .  . 
 7  .  .  .  .  .  .  .  . 


💡 Key Methods You Can Use:
   • is_valid_move(row, col) - Check if a move is legal
   • get_legal_moves() - Get all available positions
   • format_board(formatter) - Get board as string for LLM


## Examining the Simple Agent (no LLM)

Let's look at how the existing `SimpleAgent` works to understand the pattern:

In [8]:

class SimpleAgent(Agent):
    """Simple agent with basic strategy - only needs agent_id."""

    def _setup(self):
        """Setup - nothing needed for simple agent."""
        pass

    async def get_move(self, game_state: GameState) -> Tuple[int, int]:
        """Simple agent with basic strategy."""
        # Get legal moves directly from game_state

        legal_moves = []
        for row in range(game_state.board_size):
            for col in range(game_state.board_size):
                if game_state.board[row][col] == Player.EMPTY.value:
                    legal_moves.append((row, col))

        # Safety check for empty legal moves
        if not legal_moves:
            # Should not happen, but fallback to center
            center = game_state.board_size // 2
            return (center, center)

        # Try center first, otherwise random
        center = game_state.board_size // 2
        if game_state.board[center][center] == Player.EMPTY.value:
            return (center, center)

        return random.choice(legal_moves)


In [9]:
# Let's examine the SimpleAgent's strategy
simple_agent = SimpleAgent("Demo-Simple")

print("🤖 SimpleAgent Strategy Analysis:")
print("=" * 40)

# Let's make a few moves and see what it chooses
demo_game = GomokuGame(board_size=8)
demo_state = demo_game.state

print("Initial board:")
print(demo_state.format_board('standard'))

# Get SimpleAgent's first move
move = await simple_agent.get_move(demo_state.copy())
print(f"\n🎯 SimpleAgent's first move: {move}")
print(f"   Strategy: Prefers center ({demo_state.board_size//2}, {demo_state.board_size//2}) if available")

# Make that move and see the next one
demo_game.make_move(move[0], move[1])
move2 = await simple_agent.get_move(demo_game.state.copy())
print(f"\n🎯 SimpleAgent's second move: {move2}")
print(f"   Strategy: Random choice from available positions")

print("\n📝 SimpleAgent Logic Summary:")
print("   1. Try center position first")
print("   2. If center taken, choose randomly from legal moves")
print("   3. Very simple, no strategic thinking")

print("\n🔍 Board After First Step:")
print(demo_state.format_board('standard'))

🤖 SimpleAgent Strategy Analysis:
Initial board:
    0  1  2  3  4  5  6  7 
 0  .  .  .  .  .  .  .  . 
 1  .  .  .  .  .  .  .  . 
 2  .  .  .  .  .  .  .  . 
 3  .  .  .  .  .  .  .  . 
 4  .  .  .  .  .  .  .  . 
 5  .  .  .  .  .  .  .  . 
 6  .  .  .  .  .  .  .  . 
 7  .  .  .  .  .  .  .  . 


🎯 SimpleAgent's first move: (4, 4)
   Strategy: Prefers center (4, 4) if available

🎯 SimpleAgent's second move: (1, 0)
   Strategy: Random choice from available positions

📝 SimpleAgent Logic Summary:
   1. Try center position first
   2. If center taken, choose randomly from legal moves
   3. Very simple, no strategic thinking

🔍 Board After First Step:
    0  1  2  3  4  5  6  7 
 0  .  .  .  .  .  .  .  . 
 1  .  .  .  .  .  .  .  . 
 2  .  .  .  .  .  .  .  . 
 3  .  .  .  .  .  .  .  . 
 4  .  .  .  .  X  .  .  . 
 5  .  .  .  .  .  .  .  . 
 6  .  .  .  .  .  .  .  . 
 7  .  .  .  .  .  .  .  . 



## Designing Our LLM Agent

Now let's create our own LLM-powered agent! We'll build it step by step.

### What makes an LLM agent special?
- Uses natural language reasoning
- Can understand strategic concepts
- Can adapt to different situations
- Can explain its thinking process

In [11]:

import re
import json
from dotenv import load_dotenv
load_dotenv()

# First, let's design our LLM agent class structure

class StudentLLMAgent(Agent):
    """An educational LLM agent that students will build step by step."""

    def __init__(self, agent_id: str):
        super().__init__(agent_id)

    def _setup(self):
        """Setup our LLM client and prompts."""

        # We'll simulate an LLM for educational purposes
        self.llm_client = OpenAIGomokuClient(
            model="qwen/qwen-2.5-7b-instruct",
        )


    def _create_system_prompt(self, game_state, player, rival) -> str:
        """Create the system prompt that teaches the LLM how to play Gomoku."""
        return f"""
You are an expert Gomoku (Five-in-a-Row) player. You are {player}, your opponent is {rival}.
Your task is to choose exactly one best move for {player} based on the current board, following this strategies:
1. Control the center of the board early.
2. If a move creates a five-in-a-row for {player}, choose it.
3. If {rival} can win next turn (e.g., open four or equivalent threat), block it.
4. If possible, choose a move that creates two or more simultaneous winning threats (e.g., two open fours).
5. If no double threat exists, choose the move that creates the most powerful single threat, forcing {rival} to defend and setting up a future win.

Output Rules:
- The move must be on an empty square (marked as '.')
- The row and col must be valid coordinates on the board (0-indexed)
- Output only valid JSON in the exact format below.
- No explanation, reasoning, or extra text. JSON only.

Format:
```json
{{"row": <row_number>, "col": <col_number>}}
```

Examples:
```json
{{"row": 5, "col": 2}}
```
```json
{{"row": 1, "col": 2}}
```
```json
{{"row": 0, "col": 3}}
```
""".strip()

    async def get_move(self, game_state: GameState) -> Tuple[int, int]:
        """Main method: Get the next move from our LLM."""
        print(f"\n🧠 {self.agent_id} is thinking...")

        
        player = game_state.current_player.value
        rival = (Player.WHITE if self.player == Player.BLACK else Player.BLACK).value

        board_str = game_state.format_board(formatter="standard")
        board_prompt = f"Current board state:\n{board_str}\n"
        board_prompt += f"Current player: {game_state.current_player.value}\n"
        board_prompt += f"Move count: {len(game_state.move_history)}\n"
        # board_size = game_state.board_size
        if game_state.move_history:
            last_move = game_state.move_history[-1]
            board_prompt += f"Last move: {last_move.player.value} at ({last_move.row}, {last_move.col})\n"
            

        # Create messages for the LLM
        messages = [
            {"role": "system", "content": self._create_system_prompt(game_state, player, rival)},
            {"role": "user", "content": f"{board_prompt}\n\nProvide your next move as JSON without explanation."},
        ]

        print("💡 Full Prompt:\n\n")
        print(json.dumps(messages, indent=2, ensure_ascii=False))
        print()

        
        # Get response from LLM
        response = await self.llm_client.complete(messages)

        print("💡 Response:\n\n")
        print(response)
        print()

        try:
            if m := re.search(r"{[^}]+}", response, re.DOTALL):
                json_data = json.loads(m.group(0).strip())
                row, col = (json_data["row"], json_data["col"])

                if game_state.is_valid_move(row, col):
                    return (row, col)

        except json.JSONDecodeError as e:
            pass

        return self._get_fallback_move(game_state)

    def _get_fallback_move(self, game_state: GameState) -> Tuple[int, int]:
        """Simple fallback when LLM fails."""
        return game_state.get_legal_moves()[-1]

print("🎉 StudentLLMAgent class defined!")
print("   This agent demonstrates LLM-style strategic thinking.")

🎉 StudentLLMAgent class defined!
   This agent demonstrates LLM-style strategic thinking.


## Step 6: Testing Our LLM Agent

Let's create an instance of our agent and test its decision-making:

In [13]:
# Create and test our LLM agent
student_agent1 = StudentLLMAgent("StudentAI1-v1")
student_agent2 = StudentLLMAgent("StudentAI2-v1")

print("\n🧪 Testing our LLM agent's decision making...")
print("=" * 50)

# Create a test scenario
test_game = GomokuGame(board_size=8)

print("\nCurrent board:")
print(test_game.state.format_board('standard'))

# Now let our agent make a decision
print("\n🤔 Let's see what our StudentLLMAgent decides...")
student_move = await student_agent1.get_move(test_game.state.copy())
print(f"\n🎯 StudentLLMAgent chose: {student_move}")


simple_move = await student_agent2.get_move(test_game.state.copy())
print(f"🎯 SimpleAgent chose: {simple_move}")

print("\n📊 Analysis:")
print(f"   • StudentLLM1: Chose {student_move}")
print(f"   • StudentLLM2: Chose {simple_move}")
print("   • Our LLM agent shows more sophisticated decision-making!")


🧪 Testing our LLM agent's decision making...

Current board:
    0  1  2  3  4  5  6  7 
 0  .  .  .  .  .  .  .  . 
 1  .  .  .  .  .  .  .  . 
 2  .  .  .  .  .  .  .  . 
 3  .  .  .  .  .  .  .  . 
 4  .  .  .  .  .  .  .  . 
 5  .  .  .  .  .  .  .  . 
 6  .  .  .  .  .  .  .  . 
 7  .  .  .  .  .  .  .  . 


🤔 Let's see what our StudentLLMAgent decides...

🧠 StudentAI1-v1 is thinking...
💡 Full Prompt:


[
  {
    "role": "system",
    "content": "You are an expert Gomoku (Five-in-a-Row) player. You are X, your opponent is X.\nYour task is to choose exactly one best move for X based on the current board, following this strategies:\n1. Control the center of the board early.\n2. If a move creates a five-in-a-row for X, choose it.\n3. If X can win next turn (e.g., open four or equivalent threat), block it.\n4. If possible, choose a move that creates two or more simultaneous winning threats (e.g., two open fours).\n5. If no double threat exists, choose the move that creates the most p

## Step 7: Setting Up the Competition Arena

Now let's prepare for an exciting competition between our agents!

In [15]:
# Setup the competition arena
print("🏟️  Setting up the Competition Arena!")
print("=" * 50)

# Create arena with nice visualization
board_size = 8  # Smaller board for faster games
formatter = ColorBoardFormatter(board_size)
arena = GomokuArena(board_size=board_size, formatter=formatter)

print(f"⚙️  Arena Configuration:")
print(f"   📏 Board size: {board_size}x{board_size}")
print(f"   ⏱️  Time limit: 10 seconds per move")
print(f"   🎨 Visualization: Color-coded board")

# Create our competing agents
agents = {
    'student_llm1': StudentLLMAgent("StudentAI1"),
    'student_llm2': StudentLLMAgent("StudentAI2")
}

print(f"\n🤖 Competitors Ready:")
for name, agent in agents.items():
    print(f"   • {agent.agent_id} ({name})")

print("\n🎮 Ready for competition!")

🏟️  Setting up the Competition Arena!
⚙️  Arena Configuration:
   📏 Board size: 8x8
   ⏱️  Time limit: 10 seconds per move
   🎨 Visualization: Color-coded board

🤖 Competitors Ready:
   • StudentAI1 (simple)
   • StudentAI2 (student_llm)

🎮 Ready for competition!


## Step 8: The Competition - Simple vs LLM Agent

Time for the main event! Let's watch our agents compete:

In [17]:
# Single game competition
print("🥊 COMPETITION: StudentLLMAgent1 vs StudentLLMAgent2")
print("=" * 60)

# Run the game
game_result = await arena.run_game(
    agents['student_llm1'],
    agents['student_llm2'],
    verbose=True
)

print("\n" + "=" * 60)
print("🏆 GAME RESULTS")
print("=" * 60)
print(f"🥇 Winner: {game_result['winner']}")
print(f"📊 Total moves: {game_result['moves']}")
print(f"⏱️  Game duration: {game_result.get('total_time', 0):.2f} seconds")
print(f"🎯 Victory condition: {game_result['reason']}")

if game_result.get('winning_sequence'):
    print(f"🏁 Winning sequence: {len(game_result['winning_sequence'])} positions")

print("\n🎮 Final Board with Winning Sequence:")
final_display = arena.draw_board_with_winning_sequence(
    game_result['final_board'],
    game_result.get('winning_sequence', [])
)
print(final_display)

🥊 COMPETITION: StudentLLMAgent1 vs StudentLLMAgent2

BLACK's turn (StudentAI1)
    0  1  2  3  4  5  6  7 
 0  .  .  .  .  .  .  .  . 
 1  .  .  .  .  .  .  .  . 
 2  .  .  .  .  .  .  .  . 
 3  .  .  .  .  .  .  .  . 
 4  .  .  .  .  .  .  .  . 
 5  .  .  .  .  .  .  .  . 
 6  .  .  .  .  .  .  .  . 
 7  .  .  .  .  .  .  .  . 


🧠 StudentAI1 is thinking...
💡 Full Prompt:


[
  {
    "role": "system",
    "content": "You are an expert Gomoku (Five-in-a-Row) player. You are X, your opponent is O.\nYour task is to choose exactly one best move for X based on the current board, following this strategies:\n1. Control the center of the board early.\n2. If a move creates a five-in-a-row for X, choose it.\n3. If O can win next turn (e.g., open four or equivalent threat), block it.\n4. If possible, choose a move that creates two or more simultaneous winning threats (e.g., two open fours).\n5. If no double threat exists, choose the move that creates the most powerful single threat, forcing O to

## Summary

In this tutorial, you learned:

1. **Agent Architecture** - How to inherit from base classes and implement required methods
2. **Game State Analysis** - How to read and interpret game information
3. **Strategic Reasoning** - How to implement win detection, blocking, and strategic positioning
4. **LLM Integration** - How LLMs can be used for game AI (with mock implementation)
5. **Competition Framework** - How to run tournaments and analyze results
6. **Error Handling** - How to implement fallback strategies for robustness

### Key Design Patterns
- **Strategy Pattern**: Different agents implement different strategies
- **Template Method**: Base class defines structure, subclasses implement specifics  
- **Dependency Injection**: Arena accepts any agent that follows the interface
- **Fallback Pattern**: Always have a backup plan when AI fails

### Next Steps
- Experiment with different prompting strategies
- Create agents with different personalities or difficulty levels

Happy coding! 🚀